In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model 
import pickle
import pandas as pd
import numpy as np

In [3]:
## Load the ANN trained model ,scaler pickle , onehot
model=load_model('model.h5')

##load the encoder and decoder
with open('label_encoder_geo.pk1','rb') as file:
    label_encoder_geo=pickle.load(file)

with open('label_encoder_gender.pk1','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('scaler.pk1','rb') as file:
    scaler=pickle.load(file)


In [4]:
##example input data
input_data = {
    'CreditScore' : 600,
    'Geography': 'France',
    'Gender' : 'Male',
    'Age' : 40,
    'Tenure' : 3,
    'Balance' : 60000,
    'NumOfProducts' : 2,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary' : 50000
}

In [5]:
# One hot encoder 'Geography'
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\Lenovo\Downloads\ANN Classification\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [6]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [7]:
## encode categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [8]:
## concatenation one hot encoded data 
input_df=pd.concat([input_df.drop("Geography", axis=1),geo_encoded_df], axis=1)
input_df


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [10]:
## Scaling the data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [11]:
## predict churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step


array([[0.03443209]], dtype=float32)

In [12]:
prediction_prob= prediction[0][0]

In [13]:
prediction_prob

0.03443209

In [14]:
if prediction_prob > 0.5:
    print('the customer is likelt to churn')
else:
    print('the customer is not likely to churn')

the customer is not likely to churn
